In [2]:
# Data manipulation
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

In [17]:
# 1 groupby

df = pd.DataFrame({'col1': ['apple', 'banana', 'orange'] * 30,
                   'col2': np.random.rand(90),
                   'col3': np.random.randint(0, 15, 90)})

# groupby.agg(new_col_name = ('colname',agg_func on pd.Series))

(
    df
    .groupby('col1',as_index=False)
    .agg(
        sum_col2 = ('col2',np.sum), # perform aggregation
        distict_col2 = ('col2',lambda x: x.nunique()),
        collect_list_col3 = ('col3',lambda x : x.tolist()),
        collect_list_sampled_5_col3 = ('col3',lambda x : x.sample(5,random_state=42).tolist())
    )
)

,col1,sum_col2,distict_col2,collect_list_col3,collect_list_sampled_5_col3
0,apple,14.081394,30.0,"[4, 8, 11, 14, 14, 2, 11, 3, 11, 6, 9, 4, 9, 1...","[11, 11, 12, 13, 11]"
1,banana,15.594545,30.0,"[3, 8, 0, 12, 14, 5, 14, 0, 10, 4, 1, 4, 8, 14...","[12, 2, 12, 7, 10]"
2,orange,13.339478,30.0,"[13, 6, 12, 4, 8, 9, 1, 9, 7, 11, 2, 8, 13, 0,...","[6, 11, 14, 6, 7]"


In [15]:
# case when pattern

df = pd.DataFrame({'col1': ['apple', 'banana', 'orange'] * 30,
                   'col2': np.random.rand(90),
                   'col3': np.random.randint(0, 15, 90)})

df = (
    df
    .assign(
        col4 = (
           np.where(df['col2'] <= 5, '0_5',
           np.where(df['col2'] <= 15,'6_15',
           np.where(df['col2'] <= 25,'16_25',
           np.where(df['col2'] <= 50,'25_50',
           np.where(df['col2'] <= 100,'50_100',
            '100_'
                    )))))))
)

df.head()

,col1,col2,col3,col4
0,apple,0.339015,14,0_5
1,banana,0.830994,10,0_5
2,orange,0.063289,4,0_5
3,apple,0.807601,11,0_5
4,banana,0.421484,8,0_5
